In [1]:
import time
import redis
from datetime import datetime

In [2]:
from data import Market1501_Attribute

r = redis.StrictRedis(host='localhost', port=6379, db=0)
for key in r.scan_iter("*"):
    r.delete(key)

datasource = Market1501_Attribute(
    root_dir='/home/hien/Documents/datasets', download=False, extract=False, re_label_on_train=False)

start_time = time.time()

attribute_label = datasource.get_attribute('train')[1]
start_index = int(datetime.now().strftime(r'%m%d%H%M%S'))
for index in range(len(datasource.get_data('train'))):
    save_dict = {
        'path': datasource.get_data('train')[index][0],
        'person_id': datasource.get_data('train')[index][1],
        'camera_id': datasource.get_data('train')[index][2]}

    for index_attribute in range(len(attribute_label)):
        save_dict.update({attribute_label[index_attribute]: int(datasource.get_attribute(
            'train')[0][datasource.get_data('train')[index][1]][index_attribute])})
    r.hmset(start_index+index, save_dict)
    for key, value in save_dict.items():
        if key != 'path' and key != 'person_id' and key != 'camera_id':
            if value == 1:
                r.lpush('1_' + key, start_index+index)
            elif value == 0:
                r.lpush('0_' + key, start_index+index)

print("time excute: %s seconds" % (time.time() - start_time))

100%|██████████| 6738/6738 [00:00<00:00, 54534.34it/s]
Processing on query directory!
Processing on gallery directory!
100%|██████████| 39466/39466 [00:00<00:00, 91449.77it/s]
time excute: 68.17429041862488 seconds


In [6]:
from data import PA_100K

r = redis.StrictRedis(host='localhost', port=6379, db=1)
for key in r.scan_iter("*"):
    r.delete(key)

datasource = PA_100K(root_dir='/home/hien/Documents/datasets', download=False, extract=False)
all_data = datasource.get_data('train')[0] + datasource.get_data('val')[0] + datasource.get_data('test')[0]
attribute_label = datasource.get_data('train')[1]

start_time = time.time()
start_index = int(datetime.now().strftime(r'%m%d%H%M%S'))
for index in range(len(datasource.get_data('train')[0])):
        save_dict = {
            'path': datasource.get_data('train')[0][index][0]
        }
        for index_attribute in range(len(attribute_label)):
            save_dict.update({attribute_label[index_attribute]: int(datasource.get_data(
                'train')[0][index][1][index_attribute])})
        r.hmset(start_index+index, save_dict)

        for key, value in save_dict.items():
            if key != 'path':
                if value == 1:
                    r.lpush('1_' + key, start_index+index)
                elif value == 0:
                    r.lpush('0_' + key, start_index+index)
print("time excute: %s seconds" % (time.time() - start_time))

TypeError: 'NoneType' object is not subscriptable